In [2]:
%%sql

-- Capa Gold

-- 1. Preparación de los datos (staging en vista temporal)
-- Agrupo lo que viene de Silver para ver qué voy a insertar o actualizar
CREATE OR REPLACE TEMPORARY VIEW staging_metrics AS
SELECT 
    producto_id, 
    fecha_id, 
    codigo_postal_id,
    proveedor_asignado,
    estado_logistica,
    SUM(monto_linea_total) AS ingresos_totales,
    SUM(cantidad) AS unidades_vendidas,
    SUM(gasto_logistico_total) AS gasto_logistico_total,
    AVG(precio_venta_real) AS precio_promedio_unidad,
    COUNT(orden_id) AS total_pedidos
FROM lh_silver.fact_ventas
GROUP BY 
    producto_id, 
    fecha_id, 
    codigo_postal_id, 
    proveedor_asignado, 
    estado_logistica;

-- 2. Si la tabla no existe en Gold, la creo con la estructura de la vista
CREATE TABLE IF NOT EXISTS lh_gold.fact_global_negocio 
USING DELTA 
AS SELECT * FROM staging_metrics WHERE 1=0;

-- 3. Merge: Comparo por IDs y estados para no duplicar data
MERGE INTO lh_gold.fact_global_negocio AS target
USING staging_metrics AS source
ON  target.producto_id = source.producto_id 
AND target.fecha_id = source.fecha_id
AND target.codigo_postal_id = source.codigo_postal_id
AND target.proveedor_asignado = source.proveedor_asignado
AND target.estado_logistica = source.estado_logistica

WHEN MATCHED THEN
  UPDATE SET 
    target.ingresos_totales = source.ingresos_totales,
    target.unidades_vendidas = source.unidades_vendidas,
    target.gasto_logistico_total = source.gasto_logistico_total,
    target.precio_promedio_unidad = source.precio_promedio_unidad,
    target.total_pedidos = source.total_pedidos

WHEN NOT MATCHED THEN
  INSERT *;

-- 4. Optimize para buena lectura en el Direct Lake de Power BI
OPTIMIZE lh_gold.fact_global_negocio;

StatementMeta(, 17a0b496-bd91-4011-94d1-1e9ef616c6d5, 9, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1 rows and 4 fields>

<Spark SQL result set with 1 rows and 2 fields>